In [1]:
from urllib2 import Request, urlopen
import ssl
import json
import pandas as pd
import plotly.plotly as py
import cufflinks as cf
from pandas.io.json import json_normalize
import requests
from requests.packages.urllib3.exceptions import InsecureRequestWarning
import re
import os
from datetime import datetime

In [2]:
#Silent warnings of insecure request from requests library
requests.packages.urllib3.disable_warnings(InsecureRequestWarning)
pd.options.mode.chained_assignment = None  # default='warn'

In [3]:
site = 'T1_US_FNAL_Buffer'
datasets = [
    '/TT_TuneCUETP8M2T4_mtop1735_13TeV-powheg-pythia8/RunIISummer15wmLHEGS-MCRUN2_71_V1-v1/GEN-SIM',
    '/QCD_Pt-15to3000_TuneCUETP8M1_Flat_13TeV-pythia8/RunIISummer15GS-castor_MCRUN2_71_V1-v1/GEN-SIM',
    '/TT_TuneCUETP8M2T4_13TeV-powheg-pythia8-evtgen/RunIISummer15wmLHEGS-MCRUN2_71_V1-v1/GEN-SIM',
    '/TTJets_DiLept_genMET-150_TuneCUETP8M1_13TeV-madgraphMLM-pythia8/RunIISummer15wmLHEGS-MCRUN2_71_V1-v1/GEN-SIM',
    '/TTZToLLNuNu_M-10_TuneCUETP8M1_13TeV-amcatnlo-pythia8/RunIISummer15wmLHEGS-MCRUN2_71_V1_ext2-v1/GEN-SIM',
    '/ZH_HToBB_ZToNuNu_M125_13TeV_powheg_pythia8/PhaseIFall16wmLHEGS-81X_upgrade2017_realistic_v26-v1/GEN-SIM',
    '/WJetsToLNu_Wpt-200toInf_TuneCUETP8M1_13TeV-madgraphMLM-pythia8/RunIISummer15wmLHEGS-MCRUN2_71_V1_ext1-v1/GEN-SIM',
    '/DYJetsToLL_Zpt-200toInf_M-50_TuneCUETP8M1_13TeV-madgraphMLM-pythia8/RunIISummer15wmLHEGS-MCRUN2_71_V1_ext1-v1/GEN-SIM',
    '/TTGamma_Dilept_TuneCUETP8M2T4_13TeV-amcatnlo-pythia8/RunIISummer15wmLHEGS-MCRUN2_71_V1-v2/GEN-SIM',
    '/TTGamma_SingleLeptFromTbar_TuneCUETP8M2T4_13TeV-amcatnlo-fsrdown-pythia8-pythia8/RunIISummer15wmLHEGS-MCRUN2_71_V1-v1/GEN-SIM',
    '/TT_widthx4_TuneCUETP8M1_13TeV-powheg-pythia8/RunIISummer15wmLHEGS-MCRUN2_71_V1-v1/GEN-SIM',
    '/TTTo2L2Nu_noSC_TuneCUETP8M1_alphaS01273_13TeV-powheg-pythia8/RunIISummer15GS-MCRUN2_71_V1-v1/GEN-SIM',
    '/ttH_M125_13TeV_powheg_pythia8/RunIISummer15GS-MCRUN2_71_V1-v1/GEN-SIM',
    '/GJets_DR-0p4_HT-600ToInf_TuneCUETP8M1_13TeV-madgraphMLM-pythia8/RunIISummer15GS-magnetOffBS0T_MCRUN2_71_V1-v1/GEN-SIM',
    '/TT_TuneCUETP8M1_mtop1785_13TeV-powheg-pythia8/RunIISummer15GS-MCRUN2_71_V1-v1/GEN-SIM',
    '/DY1JetsToLL_M-10to50_TuneCUETP8M1_13TeV-amcatnloFXFX-pythia8/RunIISummer15GS-MCRUN2_71_V1-v1/GEN-SIM',
    '/ttHJetTobb_M125_13TeV_amcatnloFXFX_madspin_pythia8/RunIISummer15GS-MCRUN2_71_V1_ext3-v1/GEN-SIM',
    '/ST_t-channel_4f_mtop1755_leptonDecays_13TeV-amcatnlo-pythia8_TuneCUETP8M1/RunIISummer15GS-MCRUN2_71_V1-v1/GEN-SIM',
    '/MuMuG_PTG10To13000_MMuMu50_13TeV_sherpa/RunIISummer15GS-MCRUN2_71_V1-v2/GEN-SIM',
    '/GJets_DR-0p4_HT-600ToInf_TuneCUETP8M1_13TeV-madgraphMLM-pythia8/RunIISummer15GS-magnetOffBS0T_MCRUN2_71_V1-v1/GEN-SIM',
    '/WplusToENu_NNPDF30_TuneEE5C_13TeV-powheg-herwigpp/RunIISummer15wmLHEGS-MCRUN2_71_V1-v2/GEN-SIM',
    '/ST_FCNC-TA_Tleptonic_kappa_act-MadGraph5-pythia8/RunIISummer15wmLHEGS-MCRUN2_71_V1-v1/GEN-SIM',
    '/ST_tW_antitop_5f_NoFullyHadronicDecays_13TeV-powheg_TuneCUETP8M1/RunIISummer15wmLHEGS-MCRUN2_71_V1_ext1-v1/GEN-SIM',
    '/QCD_Pt_1000to1400_TuneCUETP8M1_13TeV_pythia8/PhaseIFall16GS-81X_upgrade2017_realistic_v26-v1/GEN-SIM',
    '/Vector_MonoTop_NLO_Mphi-200_Mchi-10_gSM-0p25_gDM-1p0_13TeV-madgraph/RunIISummer15wmLHEGS-MCRUN2_71_V1-v1/GEN-SIM',
    '/Vector_MonoTop_NLO_Mphi-2500_Mchi-500_gSM-0p25_gDM-1p0_13TeV-madgraph/RunIISummer15wmLHEGS-MCRUN2_71_V1-v1/GEN-SIM',
    '/WGToLNuG_PtG-500_TuneCUETP8M1_13TeV-amcatnloFXFX-pythia8/RunIISummer15wmLHEGS-MCRUN2_71_V1-v1/GEN-SIM',
    '/ZBJetsToNuNu_Zpt-200toInf_TuneCUETP8M1_13TeV-madgraphMLM-pythia8/RunIISummer15wmLHEGS-MCRUN2_71_V1_ext1-v1/GEN-SIM',
    '/BprimeBprime_M-1000_TuneCUETP8M1_13TeV-madgraph-pythia8/RunIISummer15GS-MCRUN2_71_V1-v1/GEN-SIM',
    '/ST_t-channel_antitop_4f_leptonDecays_13TeV-powheg-pythia8_TuneCUETP8M1/RunIISummer15GS-MCRUN2_71_V1-v1/GEN-SIM',
    '/ggZH_HToBB_ZToNuNu_M125_13TeV_powheg_pythia8/RunIISummer15GS-MCRUN2_71_V1_ext2-v1/GEN-SIM'
    ]

log_file = "/Users/davidurbina/Documents/stage_tests_logs/logs/T1_US_FNAL/Prod_T1_US_FNAL_Buffer/logs/mss-stage-local_littleEdit"
start_date = '2017-03-21 00:00:00'
end_date = '2017-05-02 00:00:00'
#'''


In [4]:
def get_files(site, dataset):
    url = 'https://cmsweb.cern.ch/phedex/datasvc/json/prod/filereplicas'
    params = {"node": site, "dataset": dataset}
    replicas_info = requests.get(url=url, params= params, verify=False).content
    replicas_json = json.loads(replicas_info)
    blocks_level_table = json_normalize(replicas_json['phedex']['block'])

    replicas_table = pd.DataFrame()
    for i in range(len(blocks_level_table)):
        replicas_info = json_normalize(replicas_json['phedex']['block'][i]['file'])
        replicas_table = replicas_table.append(replicas_info)
    return replicas_table

def clean_files_names (name):
    match = re.search('(/store/.+$)', name)
    if match:
        clean_name = match.group(1)
    else:
        print "warning file " + name + " not in /store/ path"
        clean_name = name
    return clean_name

def parse_executing (files_str):
    match = re.search('Executing with timeout \d+ (.+$)', files_str)
    if match:
        files_list = match.group(1).split(" ")
        files_list = files_list[1:]
        files_list = [clean_files_names(x) for x in files_list]
    return files_list

def parse_staged (action_str):
    match = re.search('^([^ ]+) STAGED', action_str)
    if match:
        file_name = clean_files_names( match.group(1) )
    else:
        print "Warning: wear STAGED line: " + action_str
        file_name = action_str
    return file_name

def num_files_executing (files_str):
    return len(parse_executing (files_str))

def files_from_test (files_str, files_test_set):
    files_list = parse_executing(files_str)
    return list(files_test_set & set(files_list))

def file_from_test (file_str, files_test_set):
      return int(file_str in files_test_set)


def num_files_from_test (files_str, files_test_set):
    files_test = files_from_test (files_str, files_test_set)
    return len(files_test)

def dataset_from_file(filename):
    if re.search('^/store/(mc|relval)', filename):
        filename_fields =  filename.split("/")[3:-2]
        filename_fields = [filename_fields[1], filename_fields[0] + '-' + filename_fields[3], filename_fields[2]]

    elif re.search('^/store/data', filename):
        filename_fields = filename.split("/")
        #print filename +  ": " + str(len(filename.split("/")))
        if (len(filename_fields) == 12):
            filename_fields =  filename_fields[3:-5]
            filename_fields = [filename_fields[1], filename_fields[0] + '-' + filename_fields[3], filename_fields[2]]
        elif(len(filename_fields) == 11):
            filename_fields =  filename_fields[3:-4]
            filename_fields = [filename_fields[1], filename_fields[0] + '-' + filename_fields[3], filename_fields[2]]
        elif (len(filename_fields) == 9):
            filename_fields =  filename_fields[3:-2]
            filename_fields = [filename_fields[1], filename_fields[0] + '-' + filename_fields[3], filename_fields[2]]
        else:
            print "Warning, file: " + filename + " has a not expected number of fields " + str(len(filename_fields))

    elif re.search('^/store/results', filename):
        filename_fields =  filename.split("/")[4:-3]
        filename_fields = [filename_fields[0], filename_fields[1] , filename_fields[-1]]

    else:
        print "Warning, filename: " + filename + " is not belongs to data or mc"
        return None
    dataset = '/' + '/'.join(filename_fields)
    return dataset

In [5]:
files_test = []
for dataset in datasets:
    replicas_table = get_files(site, dataset)
    file_names = replicas_table['name'].tolist()
    for f in file_names:
        files_test.append(f)
files_test_set = set(files_test)

start_date = datetime.strptime(start_date,'%Y-%m-%d %H:%M:%S')
end_date = datetime.strptime(end_date,'%Y-%m-%d %H:%M:%S')


with open(log_file) as f:
    data = f.readlines()

count = 0
df_count =  0
dfs_list = []
df_tmp = pd.DataFrame(index= range(0, 1000), columns=['datetime', 'fileStager_ID', 'action'])
for line in data:
    match = re.search('(^\d+-\d+-\d+ \d+:\d+:\d+): FileStager\[(\d+)\]: (.+$)', line)
    if match:
        datetime_field = match.group(1)
        datetime4compare = datetime.strptime(datetime_field,'%Y-%m-%d %H:%M:%S')
        filestagerID_field = match.group(2)
        action_field = match.group(3)
    #This section asumes that the log is sorted by datetime
    if (datetime4compare < start_date):
        continue
    elif (datetime4compare > end_date):
        break
    else:
        if (count%1000 == 0 and count != 0):
            dfs_list.append(df_tmp)
            df_count += 1
            df_tmp = pd.DataFrame(index=range(count,count+1000), columns=['datetime', 'fileStager_ID', 'action'])

        df_tmp.loc[count] = [datetime_field, filestagerID_field, action_field]
        #print str(count) + "\r",
        count += 1
        if count == len(data):
            dfs_list.append(df_tmp.dropna())


data_df = pd.concat(dfs_list)
data_df['datetime'] = pd.to_datetime(data_df['datetime'])


In [7]:
#Executing events
executing_df = data_df[data_df['action'].str.contains("Executing")]
executing_df['count'] = executing_df['action'].apply(num_files_executing)
executing_df['count_test'] = executing_df['action'].apply(num_files_from_test,files_test_set = files_test_set)
executing_df['count_others'] = executing_df['count'] - executing_df['count_test']

executing_df['files_list'] = executing_df['action'].apply(parse_executing)
files_series = executing_df.apply(lambda x: pd.Series(x['files_list']),axis=1).stack().reset_index(level=1, drop=True)
files_series.name = 'file'
executing_df_info_by_file = executing_df[['datetime', 'fileStager_ID']].join(files_series)
executing_df_info_by_file['dataset'] =  executing_df_info_by_file['file'].apply(dataset_from_file)
executing_df_info_by_file['count_test'] = executing_df_info_by_file['file'].apply(file_from_test, files_test_set=files_test_set)
executing_df_info_by_file_test_only = executing_df_info_by_file[executing_df_info_by_file['count_test'] > 0]
executing_df_info_by_file_test_only.index = range(0,len(executing_df_info_by_file_test_only))
executing_df_info_by_file_test_only_pivot = executing_df_info_by_file_test_only.pivot(columns='dataset', values='count_test')
executing_df_info_by_file_test_only_pivot['datetime'] = executing_df_info_by_file_test_only['datetime']

datasets_rename = range(1,len(executing_df_info_by_file_test_only_pivot.columns))
datasets_rename= [str(x) for x in datasets_rename]
datasets_rename.append('datetime')
datasets_rename
executing_df_info_by_file_test_only_pivot.columns = datasets_rename
executing_df_info_by_file_test_only_pivot_indexed = executing_df_info_by_file_test_only_pivot.groupby('datetime')[datasets_rename[:-1]].sum()
executing_df_info_by_file_test_only_pivot_indexed_minutes = executing_df_info_by_file_test_only_pivot_indexed.resample("1Min").sum().dropna(how='all')
executing_df_info_by_file_test_only_pivot_indexed_hours = executing_df_info_by_file_test_only_pivot_indexed.resample("1H").sum().dropna(how='all')


In [8]:
executing_df_info_by_file_test_only_pivot_indexed_minutes.iplot(kind='scatter', filename = "executing_scatter_perDatasets_minutes_" + site, xTitle = "date-time", yTitle = "Number of files", title = "Executing action for " + site + " (grouped by minutes for each test dataset [bin size = 1 Min] )" )

In [9]:
executing_df_info_by_file_test_only_pivot_indexed_hours.iplot(kind='scatter', filename = "executing_scatter_perDatasets_hours_" + site, xTitle = "date-time", yTitle = "Number of files", title = "Executing action for " + site + " (grouped by hours for each test dataset [bin size = 1Hr] )" )


In [10]:
count_executing = executing_df_info_by_file_test_only['file'].value_counts().value_counts().sort_index()
print (count_executing.to_string())

1        68
2      1950
3       711
4      7128
5      2602
6      2574
7      1370
8      1553
9      1465
10     1211
11     1255
12      799
13      585
14      523
15      604
16      524
17      656
18      387
19      450
20      358
21      249
22      518
23      609
24      220
25      270
26      120
27      159
28      432
29      299
30      398
31      233
32      216
33      253
34      312
35      139
36      121
37      172
38      120
39      129
40       79
41       57
42       34
43       22
44       32
45      133
46       24
47       56
48       55
49       18
50      103
51       43
52      106
53       78
54       69
55       27
56       19
57       24
58       46
59       21
60       18
61        8
62       11
63        6
64       10
65       16
66        3
67        6
68        6
69       11
70        4
71       31
72       10
73        1
74       11
75       58
76       10
77        3
78        2
79       15
80        1
84        6
85       11
89        1
90  

In [11]:
executing_df_indexed =  executing_df.groupby('datetime')['count_test','count_others'].sum()
executing_df_indexed_minutes = executing_df_indexed.resample("1Min").sum().dropna()
executing_df_indexed_hours = executing_df_indexed.resample("1H").sum().dropna()

In [12]:
executing_df_indexed_minutes.iplot(kind='bar', filename = "executing_bar_min_" + site, xTitle = "date-time", yTitle = "Number of files", title = "Executing actions for " + site + " (grouped by minutes [bin size = 1 Min] )"  )


In [13]:
executing_df_indexed_hours.iplot(kind='bar', filename = "executing_bar_hours_" + site , xTitle = "date-time", yTitle = "Number of files", title = "Executing actions for " + site + " (grouped by hours [bin size = 1 Hr] )" )


In [14]:
#STAGED
staged_df = data_df[data_df['action'].str.contains("STAGED")]
staged_df['file'] = staged_df['action'].apply(parse_staged)
staged_df['dataset'] = staged_df['file'].apply (dataset_from_file)
staged_df['count'] = 1
staged_df['count_test'] = staged_df['file'].apply(file_from_test, files_test_set=files_test_set)
staged_df['count_others'] = staged_df['count'] - staged_df['count_test']

staged_test_only_df = staged_df[staged_df['count_test'] > 0]
staged_test_only_df_pivot = staged_test_only_df.pivot(columns='dataset', values='count_test')
staged_test_only_df_pivot['datetime'] = staged_test_only_df['datetime']

for idx, dataset in enumerate(staged_test_only_df_pivot.columns):
    print (str(idx+1)  + " => " + dataset)

datasets_rename = range(1,len(staged_test_only_df_pivot.columns))
datasets_rename= [str(x) for x in datasets_rename]
datasets_rename.append('datetime')
staged_test_only_df_pivot.columns = datasets_rename
staged_test_only_df_pivot_indexed = staged_test_only_df_pivot.groupby('datetime')[datasets_rename[:-1]].sum()
staged_test_only_df_pivot_indexed_minutes = staged_test_only_df_pivot_indexed.resample("1Min").sum().dropna(how='all')
staged_test_only_df_pivot_indexed_hours = staged_test_only_df_pivot_indexed.resample("1H").sum().dropna(how='all')

1 => /BprimeBprime_M-1000_TuneCUETP8M1_13TeV-madgraph-pythia8/RunIISummer15GS-MCRUN2_71_V1-v1/GEN-SIM
2 => /DY1JetsToLL_M-10to50_TuneCUETP8M1_13TeV-amcatnloFXFX-pythia8/RunIISummer15GS-MCRUN2_71_V1-v1/GEN-SIM
3 => /DYJetsToLL_Zpt-200toInf_M-50_TuneCUETP8M1_13TeV-madgraphMLM-pythia8/RunIISummer15wmLHEGS-MCRUN2_71_V1_ext1-v1/GEN-SIM
4 => /GJets_DR-0p4_HT-600ToInf_TuneCUETP8M1_13TeV-madgraphMLM-pythia8/RunIISummer15GS-magnetOffBS0T_MCRUN2_71_V1-v1/GEN-SIM
5 => /MuMuG_PTG10To13000_MMuMu50_13TeV_sherpa/RunIISummer15GS-MCRUN2_71_V1-v2/GEN-SIM
6 => /QCD_Pt-15to3000_TuneCUETP8M1_Flat_13TeV-pythia8/RunIISummer15GS-castor_MCRUN2_71_V1-v1/GEN-SIM
7 => /QCD_Pt_1000to1400_TuneCUETP8M1_13TeV_pythia8/PhaseIFall16GS-81X_upgrade2017_realistic_v26-v1/GEN-SIM
8 => /ST_FCNC-TA_Tleptonic_kappa_act-MadGraph5-pythia8/RunIISummer15wmLHEGS-MCRUN2_71_V1-v1/GEN-SIM
9 => /ST_t-channel_4f_mtop1755_leptonDecays_13TeV-amcatnlo-pythia8_TuneCUETP8M1/RunIISummer15GS-MCRUN2_71_V1-v1/GEN-SIM
10 => /ST_t-channel_antitop_4

In [15]:
staged_test_only_df_pivot_indexed_minutes.iplot(kind='scatter', filename = "staged_scatter_perDatasets_minutes_" + site, xTitle = "date-time", yTitle = "Number of files", title = "STAGED action for " + site + " (grouped by minutes for each test dataset [bin size = 1 Min] )" )


The draw time for this plot will be slow for all clients.


/Users/davidurbina/anaconda/lib/python2.7/site-packages/plotly/api/v1/clientresp.py:40: UserWarning:

Estimated Draw Time Too Long



In [16]:
staged_test_only_df_pivot_indexed_minutes.iplot(kind='scatter', filename = "staged_scatter_perDatasets_minutes_" + site, xTitle = "date-time", yTitle = "Number of files", title = "STAGED action for " + site + " (grouped by minutes for each test dataset [bin size = 1 Min] )" )


The draw time for this plot will be slow for all clients.


In [17]:
staged_df_indexed = staged_df.groupby('datetime')['count_test', 'count_others'].sum()
staged_df_indexed_minutes = staged_df_indexed.resample("1Min").sum().dropna()
staged_df_indexed_hours = staged_df_indexed.resample("1H").sum().dropna()


In [18]:
staged_df_indexed_minutes.iplot(kind='bar', filename = "staged_bar_min_" + site, xTitle = "date-time", yTitle = "Number of files", title = "STAGED action for " + site + " (grouped by minutes [bin size = 1 Min] )" )


In [19]:
staged_df_indexed_hours.iplot(kind='bar', filename = "staged_bar_hours_" + site, xTitle = "date-time", yTitle = "Number of files", title = "STAGED action for " + site + " (grouped by hours [bin size = 1 Hr] )" )


In [20]:
count_staging = staged_test_only_df['file'].value_counts().value_counts().sort_index()
print (count_staging.to_string())


1     3196
2    53372
3       29
